In [77]:
import cv2
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [108]:
class Preprocess():
    def __init__(self,image,gamma=0.3):
        self.image = image
        self.gamma = gamma
    
    def adjust_gamma(self):
        invGamma = 1.0 / self.gamma
        table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        return cv2.LUT(self.image, table)

    def sharpening(self,image):
        smoothed = cv2.GaussianBlur(image,(3,3),5)
        unsh = cv2.addWeighted(image,1.8,smoothed,-0.5,0)
        return unsh


    def resizer(self,image):
        img_re = image
        scale_percent = 160 # percent of original size
        width = int(img_re.shape[1] * scale_percent / 100)
        height = int(img_re.shape[0] * scale_percent / 100)
        dim = (width, height)
        resized = cv2.resize(img_re, dim, interpolation = cv2.INTER_CUBIC)
        return resized

    def Filter(self):
        image_gamma = self.adjust_gamma()
        image_sharp = self.sharpening(image_gamma)
        resized = self.resizer(image_sharp)
        return resized



In [81]:
img = cv2.imread('Images/letras.png')
#img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#equ = cv2.equalizeHist(img)
#cv2.imwrite('HistEqua.png',equ)

## Retirar sombra
rgb_planes = cv2.split(img)
result_planes = []
result_norm_planes = []
dst = np.zeros(shape=(5,2))
for plane in rgb_planes:
    dilated_img = cv2.dilate(plane, np.ones((13,13), np.uint8))
    bg_img = cv2.medianBlur(dilated_img,21)
    diff_img = 255 - cv2.absdiff(plane, bg_img)
    norm_img = cv2.normalize(diff_img, dst ,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8UC1)
    result_planes.append(diff_img)
    result_norm_planes.append(norm_img)

result = cv2.merge(result_planes)
result_norm = cv2.merge(result_norm_planes)
cv2.imwrite('NoShaw.png',result_norm)

def adjust_gamma(image):
    invGamma = 1.0 / 0.3
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image,table)

gamma_corection = adjust_gamma(result_norm)
cv2.imwrite('gamma_corection.png',gamma_corection)




True

In [110]:
image = gamma_corection
img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#Thres
ret,thr= cv2.threshold(img_gray,100,255,cv2.THRESH_BINARY)
# Erosion
kernel = np.ones((3,3), np.uint8)
img_erosion = cv2.erode(thr, kernel, iterations=1)

preproce = Preprocess(img_erosion)
img_erosion = preproce.sharpening(img_erosion)

cv2.imwrite('letras2.png',img_erosion)


True

In [84]:
# Find the Edges:
res = cv2.findContours(img_erosion.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
if cv2.__version__.split(".")[0] == '3':
    im2, contours, hierarchy = res
else:
    contours, hierarchy = res

rects = []
for c in contours:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    x, y, w, h = cv2.boundingRect(approx)
    if h >= 16:
        # if height is enough
        # create rectangle for bounding
        rect = [x, y, w, h]
        rects.append(rect)
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 1);

cv2.imwrite('letras_final.png', img)
print(rects)

[[69, 83, 19, 23], [287, 82, 22, 27], [198, 81, 17, 23], [151, 82, 20, 22], [26, 81, 18, 27], [325, 80, 17, 31], [238, 81, 31, 22], [358, 78, 26, 26], [109, 78, 17, 26], [589, 35, 19, 28], [629, 34, 18, 29], [545, 32, 18, 27], [514, 31, 26, 27], [375, 30, 8, 26], [403, 29, 18, 24], [243, 29, 16, 25], [110, 29, 18, 21], [69, 29, 20, 20], [478, 28, 22, 28], [153, 28, 22, 24], [201, 27, 18, 27], [444, 26, 18, 31], [331, 26, 21, 25], [286, 25, 24, 23], [23, 23, 19, 27], [0, 0, 666, 147]]


# New Class Image Process

In [98]:
NI = 0
path = 'ResultLetter/'
for posi in rects:
    x,y,w,h = (posi[0],posi[1],posi[2],posi[3])
    #print(x,y,w,h)
    letter_img = img_erosion[y:y+h,x:x+w]
    # Interpolation:
    preproce = Preprocess(img_erosion)
    letter_img = preproce.resizer(letter_img)
    cv2.imwrite(path+'Letter'+str(NI)+'.png',letter_img)
    NI+=1
